# Práctica 1 Sistemas inteligentes Linja
## Hecho por César Rodríguez Villagrá

En esta práctica se hace la implementación del juego de linja en el que está implementado minimax para dar "inteligencia" al ordenador.


## Implementación del juego linja:
El código necesario para ejecutar el juego Linja:
### Imports necesarios para la práctica:


In [ ]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import display, clear_output
from ipywidgets import HTML
from time import sleep, time
import math

### Clase del juego Linja:

In [ ]:
class Tablerolinja:
    # Negras(persona) empiezan arriba, Rojas(ordenador) abajo
    def __init__(self, matrix: List[List]) -> None:
        """Inicializa el tablero de Linja

        Args:
            matrix (List[List]): matriz que se va a usar como tablero
        """
        self.setMatrix(matrix)
        self.turno = 1
        self.nmov = 1
        self.nAvan = 1

    def __eq__(self, other) -> bool:
        """Función que determina que dos estados del tablero de Linja son iguales

        Args:
            other (Tablerolinja): El elemento con el que lo queremos comparar

        Returns:
            bool: True si son iguales, false si no
        """
        m1 = self.getMatrix()
        m2 = other.getMatrix()
        for i in range(len(m1)):
            for j in range(len(m1[i])):
                if (m1[i][j] != m2[i][j]):
                    return False
        return True

    def getCodigoTablero(caracter: str) -> int:
        """Devuelve el código de mapa asociado a los caracteres definidos.

        Args:
            caracter (str): El caracter a codificar

        Returns:
            int: El entero codificado del caracter pasado
        """
        codigo = 0
        if caracter == 'V':
            codigo = 0
        if caracter == 'N':
            codigo = 1
        elif caracter == 'R':
            codigo = 2
        return codigo

    def codificar(self) -> None:
        """Codifica la matriz actual de caracteres a enteros del 0 al 2
        """
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[i])):
                if self.matrix[i][j] == "V":
                    self.matrix[i][j] = 0
                elif self.matrix[i][j] == "N":
                    self.matrix[i][j] = 1
                elif self.matrix[i][j] == "R":
                    self.matrix[i][j] = 2

    def setMatrix(self, matrix: List[List]) -> None:
        """Añade la matriz de datos al tablero

        Args:
            matrix (List[List]): La matriz que queremos introducir como nuevo tablero
        """
        self.matrix = deepcopy(matrix)

    def getMatrix(self) -> List[List]:
        """Devuelve una copia de la matriz del tablero de la clase

        Returns:
            List[List]: Una copia de la matriz del tablero
        """
        return deepcopy(self.matrix)

    def placeTile(self, row: int, col: int, tile: int) -> None:
        """Pone una ficha en la posicion del tablero pasada

        Args:
            row (int): fila
            col (int): columna
            tile (int): ficha a poner
        """
        self.matrix[row][col] = tile

    def deleteTile(self, row: int, col: int) -> None:
        """Borra el contenido de una celda del tablero

        Args:
            row (int): la fila
            col (int): la columna
        """
        self.placeTile(row, col, 0)

    def utility(self) -> int:
        """Función de utilidad para el juego

        Returns:
            int: El valor de utilidad del tablero
        """
        return self.utilityroja()-self.utilitynegra()

    def utilityroja(self) -> int:
        """Calcula la utilidad de las fichas rojas

        Returns:
            int: el valor de la utilidad
        """
        puntuacion = [5, 3, 2, 1, 0, 0, 0, 0]
        puntos = 0
        i = 0
        j = 0
        tablero = self.getMatrix()
        for i in range(len(tablero)):
            for j in range(len(tablero[i])):
                if tablero[i][j] == 2:
                    puntos = puntos+puntuacion[i]
        return puntos

    def utilitynegra(self) -> int:
        """Calcula la utilidad de las fichas negras

        Returns:
            int: el valor de la utilidad
        """
        puntuacion = [0, 0, 0, 0, 1, 2, 3, 5]
        puntos = 0
        i = 0
        j = 0
        tablero = self.getMatrix()
        for i in range(len(tablero)):
            for j in range(len(tablero[i])):
                if tablero[i][j] == 1:
                    puntos = puntos+puntuacion[i]
        return puntos

    def estaLibre(self, pos: List[int]) -> int:
        """Nos devuleve si una fila determinada está libre o no

        Args:
            pos (List[int]): la fila que queremos comprobar

        Returns:
            int: -1 si no está libre y otro número es la primera posición libre de la fila
        """
        for i in range(len(pos)):
            if (pos[i] == 0):
                return i
        return -1

    def numMovs(self, row: int) -> int:
        """Calcula el número de movimientos del segundo turno que corresponden

        Args:
            row (int): la fila

        Returns:
            int: el número de movimientos
        """
        numfichas = 0
        for i in range(0, len(self.matrix[row])):
            if (self.matrix[row][i] != 0):
                numfichas = numfichas + 1
        return numfichas

    def obtenerMov(self, row: int, col: int, playerID: int, numMov: int, numAvan: int, coldest: int = None) -> List[List]:
        """Obtiene el movimiento posible para las condiciones dadas

        Args:
            row (int): la fila origen
            col (int): la columna origen
            playerID (int): el jugador actual
            numMov (int): el número de movimientos
            numAvan (int): el número de fichas que avanza
            coldest (int, optional): Columna de destino. Por defecto None.

        Returns:
            List[List]: La matriz del tablero después de hacer el movimmiento, si no se puede hacer el movimietno devuleve la matriz del inicio sin cambios
        """
        aux = Tablerolinja(self.getMatrix())
        if (self.matrix[row][col] != playerID):
            return aux.getMatrix()
        if (playerID == 1):
            plyr = 1
        elif (playerID == 2):
            plyr = -1
        if (numMov == 1):
            if (row + plyr >= len(aux.matrix) or row + plyr < 0):
                return aux.getMatrix()
            if (coldest == None):
                pos = aux.estaLibre(aux.matrix[row + plyr])
            else:
                pos = coldest
            if (pos != -1):
                aux.placeTile(row + plyr, pos, playerID)
                aux.placeTile(row, col, 0)
            return aux.getMatrix()
        elif (numMov == 2):
            if (row + plyr*numAvan >= len(aux.matrix) or row + plyr*numAvan < 0):
                return aux.getMatrix()
            if (coldest == None):
                pos = aux.estaLibre(aux.matrix[row + plyr*numAvan])
            else:
                pos = coldest
            if (pos != -1):
                aux.placeTile(row + (plyr*numAvan), pos, playerID)
                aux.placeTile(row, col, 0)
            return aux.getMatrix()
        return aux.getMatrix()

    def obtenerFichasMovibles(self, playerID: int, numMov: int, numAvan: int) -> list[tuple]:
        """Obtiene todas las fichas movibles en este instante

        Args:
            playerID (int): El jugador actual
            numMov (int): El numero de movimientos que se puede hacer
            numAvan (int): El numero de fichas que puede avanzar

        Returns:
            list[tuple]: Devuleve una lista de tuplas en la que continen las coordenadas de las fichas con movimientos posibles
        """
        fichasAMover = []
        if (self.fin() == True):
            return fichasAMover
        if numMov == 1:
            for i in range(0, len(self.matrix)):
                for j in range(0, len(self.matrix[i])):
                    if self.matrix[i][j] == playerID:
                        if self.obtenerMov(i, j, playerID, 1, 1) != self.getMatrix():
                            fichasAMover.append((i, j))
        elif numMov == 2:
            for i in range(0, len(self.matrix)):
                for j in range(0, len(self.matrix[i])):
                    if self.matrix[i][j] == playerID:
                        if self.obtenerMov(i, j, playerID, 2, numAvan) != self.getMatrix():
                            fichasAMover.append((i, j))
        return fichasAMover

    def cambioTurno(self) -> None:
        """Cambia el turno actual
        """
        self.nmov = 1
        self.nAvan = 1
        if (self.turno == 1):
            self.turno = 2
        else:
            self.turno = 1

    def turnoATexto(self, turno: int = None) -> str:
        """Devuelve el turno actual en texto

        Args:
            turno (int, optional): El turno del que se quiere obtener el texto. Por defecto None.

        Returns:
            str: El texto correspondiente al turno
        """
        textos_turnos = {
            1: "Fichas Negras",
            2: "Fichas Rojas",
        }
        if turno is None:
            return textos_turnos[self.turno]
        else:
            return textos_turnos[turno]

    def hacerMov1(self, fila: int, col: int, coldest: int = None) -> None:
        """Realiza el movimiento 1 del jugador

        Args:
            fila (int): fila
            col (int): columna
            coldest (int, optional): Columna de destino. Por defecto None.
        """
        if (coldest != None):
            self.setMatrix(self.obtenerMov(
                fila, col, self.turno, 1, 1, coldest=coldest))
        else:
            self.setMatrix(self.obtenerMov(fila, col, self.turno, 1, 1))
        if (self.turno == 1):
            plyr = 1
        else:
            plyr = -1
        self.nAvan = self.getNFichasEnFila(fila+plyr)
        self.nmov = 2

    def hacerMov2(self, fila: int, col: int, nmovs: int, coldest: int = None) -> None:
        """Realiza el movimiento 2 del jugador

        Args:
            fila (int): fila
            col (int): columna
            nmovs (int): nº de movmimentos posibles
            coldest (int, optional): Columna de destino. Por defecto None.
        """
        if (coldest != None):
            self.setMatrix(self.obtenerMov(
                fila, col, self.turno, 2, nmovs,  coldest=coldest))
        else:
            self.setMatrix(self.obtenerMov(fila, col, self.turno, 2, nmovs))

        self.nmov = 1
        self.nAvan = 1

    def getNFichasEnFila(self, fila: int) -> int:
        """Obtiene el número de fichas en una fila determinada

        Args:
            fila (int): la fila

        Returns:
            int: el nº de fichas en la fila
        """
        contador = -1
        for celda in self.matrix[fila]:
            if celda != 0:
                contador = contador+1
        return contador

    def encontrarPlyrArriba(self) -> int:
        """Devuleve el número del jugador con la ficha más arriba del tablero

        Returns:
            int: El jugador
        """
        filas = len(self.matrix)
        columnas = len(self.matrix[0])
        for i in range(filas):
            for j in range(columnas):
                if (self.matrix[i][j] != 0):
                    playerarriba = self.matrix[i][j]
                    return playerarriba

    def fin(self) -> bool:
        """Determina si el juego ha finalizado

        Returns:
            bool: True si ha finalizado, False si no
        """
        playerarriba = self.encontrarPlyrArriba()
        nfichasplrarrib = 0
        if playerarriba == 1:
            playerabajo = 2
        elif playerarriba == 2:
            playerabajo = 1
        for fila in self.matrix:
            nfichasplrarrib += fila.count(playerarriba)
        for fila in self.matrix:
            nfichasplrarrib -= fila.count(playerarriba)
            if fila.count(playerabajo) != 0:
                if nfichasplrarrib == 0:
                    return True
                else:
                    return False

    def obtenerGanador(self) -> str:
        """Devuelve el jugador que ha ganado la partida

        Returns:
            str: El tetxo con el jugador que ha ganado
        """
        if self.utilitynegra() > self.utilityroja():
            return self.turnoATexto(1)
        else:
            return self.turnoATexto(2)

    def entrada(self) -> str:
        """Funcion para manejar la entrada del usuario y la máquina

        Returns:
            str: si alguna de las entradas es "salir", lo devuleve para que en el bucle donde llame la funcion pare la ejecución
        """
        salida = "salir"
        if (self.turno == 1):
            print("Jugador negro (ordenador) = 1, Jugador rojo = 2")
            print("Ahora le toca al jugador: ", self.turno)
            while (self.nmov == 1):
                print("Fichas posibles a mover:",
                      self.obtenerFichasMovibles(self.turno, 1, 1))
                clear_output()
                display(HTML(self.get_html()))
                filain = input("Fila: ")
                if (filain == salida):
                    return filain
                fila = int(filain)
                while (fila < 0 or fila >= len(self.matrix)-1):
                    print("Error en la entrada de la fila, introduce una válida")
                    filain = input("Fila: ")
                    if (filain == salida):
                        return filain
                    fila = int(filain)
                columnain = input("Columna: ")
                if (columnain == salida):
                    return columnain
                columna = int(columnain)
                while (columna < 0 or columna >= len(self.matrix[fila])):
                    print(
                        "Error en la entrada de la columna, introduce una válida")
                    columnain = input("Columna: ")
                    if (columnain == salida):
                        return columnain
                    columna = int(columnain)
                print("Entrada fila ", fila, " y columna", columna)
                if (self.matrix[fila][columna] == self.turno):
                    print("Se moverá la ficha de la fila",
                          fila, "y columna", columna)
                    columnadin = input("Columna destino: ")
                    if (columnadin == salida):
                        return columnadin
                    columnad = int(columnadin)
                    while (columnad < 0 or columnad >= len(self.matrix[fila]) or self.matrix[fila+1][columnad] != 0):
                        print(
                            "Error en la entrada de la columna destino, introduce una válida")
                        columnadin = input("Columna destino: ")
                        if (columnadin == salida):
                            return columnadin
                        columnad = int(columnadin)
                    self.hacerMov1(fila, columna, coldest=columnad)

                elif self.matrix[fila][columna] == 0:
                    print(
                        "La posición en la que estás intentando mover está vacía, intententalo de nuevo")
                    sleep(0.5)
                else:
                    print(
                        "Estás intentando mover una ficha que no es tuya, intententalo de nuevo")
                    sleep(0.5)
            while (self.nmov == 2):
                print("Segundo movimiento del turno.")
                print("Fichas posibles a mover", self.nAvan, "posiciones",
                      self.obtenerFichasMovibles(self.turno, 2, self.nAvan))
                clear_output()
                display(HTML(self.get_html()))
                if (self.nAvan != 0):
                    filain = input("Fila: ")
                    if (filain == salida):
                        return filain
                    fila = int(filain)
                    while (fila < 0 or fila >= len(self.matrix)-self.nAvan):
                        print("Error en la entrada de la fila, introduce una válida")
                        filain = input("Fila: ")
                        if (filain == salida):
                            return filain
                        fila = int(filain)
                    columnain = input("Columna: ")
                    if (columnain == salida):
                        return columnain
                    columna = int(columnain)
                    while (columna < 0 or columna >= len(self.matrix[fila])):
                        print(
                            "Error en la entrada de la columna, introduce una válida")
                        columnain = input("Columna: ")
                        if (columnain == salida):
                            return columnain
                        columna = int(columnain)
                    print("Entrada fila ", fila, " y columna", columna)
                    if (self.matrix[fila][columna] == self.turno):
                        print("Se moverá la ficha de la fila",
                              fila, "y columna", columna)

                        columnadin = input("Columna destino: ")
                        if (columnadin == salida):
                            return columnadin
                        columnad = int(columnadin)
                        while (columnad < 0 or columnad >= len(self.matrix[fila+self.nAvan]) or self.matrix[fila+self.nAvan][columnad] != 0):
                            print(
                                "Error en la entrada de la columna destino, introduce una válida")
                            columnadin = input("Columna destino: ")
                            if (columnadin == salida):
                                return columnadin
                            columnad = int(columnadin)
                        self.hacerMov2(
                            fila, columna, self.nAvan, coldest=columnad)
                        self.cambioTurno()
                        clear_output()
                        display(HTML(self.get_html()))

                    elif self.matrix[fila][columna] == 0:
                        print(
                            "La posición en la que estás intentando mover está vacía, intententalo de nuevo")
                        sleep(0.5)
                    else:
                        print(
                            "Estás intentando mover una ficha que no es tuya, intententalo de nuevo")
                        sleep(0.5)
                else:
                    self.cambioTurno()
                    clear_output()
                    display(HTML(self.get_html()))
        else:
            print("Jugador negro (ordenador) = 1, Jugador rojo = 2")
            print("Ahora le toca al jugador: ", self.turno)
            self.nmov = 1
            self.setMatrix(performActionMinMax(self, 2).getMatrix())
            self.cambioTurno
            self.cambioTurno()

    def entradaAuto(self) -> None:
        """Funcion similar a entrada pero para manejar la partida entre una máquina contra otra
        """
        if (self.turno == 1):
            print("Jugador negro (ordenador) = 1, Jugador rojo = 2")
            print("Ahora le toca al jugador: ", self.turno)
            display(HTML(self.get_html()))
            self.setMatrix(performActionMinMax(self, 1).getMatrix())
            self.cambioTurno()
        else:
            print("Jugador negro (ordenador) = 1, Jugador rojo = 2")
            print("Ahora le toca al jugador: ", self.turno)
            display(HTML(self.get_html()))
            self.setMatrix(performActionMinMax(self, 2).getMatrix())
            self.cambioTurno()

    def moveCanBeMade(self, player: int) -> bool:
        """Nos dice si el jugador puede hacer algún movimiento

        Args:
            player (int): el jugador

        Returns:
            bool: True si puede hacer movimiento, false si no
        """
        fichas = self.obtenerFichasMovibles(self.turno, self.nmov, self.nAvan)
        if len(fichas) == 0:
            return False
        else:
            return True

    def get_content(self, coord) -> list[None]:
        """
        Obtiene el contenido de una determinada posición.

        Parameters
        ----------
        coord : Posición [y,x] de la que queremos conocer el contenido

        Returns
        --------
        contenido : Una lista de tamaño 1 .
        """
        contenido = [None]

        if self.matrix[coord[0]][coord[1]] == 0:
            contenido[0] = "casillavacia"
        elif self.matrix[coord[0]][coord[1]] == 1:
            contenido[0] = "casillanegra"
        elif self.matrix[coord[0]][coord[1]] == 2:
            contenido[0] = "casillaroja"

        return contenido

    def get_html(self) -> str:
        """ Muestra una representación gráfica del juego.

        Devuelve un "string" que contiene HTML para poder visualizarlo

        """
        element_image = {
            "casillavacia": "./ImagenesCasillasLinja/CasillaVacia.png",
            "casillanegra": "./ImagenesCasillasLinja/CasillaNegra.png",
            "casillaroja": "./ImagenesCasillasLinja/casillaRoja.png"
        }

        height = len(self.matrix)
        width = len(self.matrix[0])
        html_string = ("<style> img.game {width: 87px !important; height: 65px !important;}</style>"
                       "<style> table {max-width: 800px; width: 100%; border-collapse: collapse;} td {padding: 8px; text-align: left;}</style>"
                       "<h1 style='text-align: center; max-width: 800px;'>Linja</h1>"
                       f"<p>Turno del jugador: {self.turno} ({self.turnoATexto()})</p>"
                       f"<p>Movimiento: {self.nmov}/2 del turno, moviendo {self.nAvan} posiciones</p>"
                       f"<p>Fichas movibles: {self.obtenerFichasMovibles(self.turno, self.nmov, self.nAvan)}</p>"
                       "<table style='width: 100%;'>"
                       "<tr>"
                       f"<td>Puntuación rojas: {self.utilityroja()}</td>"
                       f"<td>Puntuación negras: {self.utilitynegra()}</td>"
                       "</tr>"
                       f"<td colspan='2' style='text-align: center;'> Utilidad para el jugador 1: {-self.utility()}</td>"
                       "</tr>"
                       "<table>")

        new_row = "<tr>"
        end_row = "</tr>"
        html_string += new_row
        html_string += "<td>+</td>"
        for j in range(width):
            html_string += f"<td style='text-align: center;'>{j}</td>"
        html_string += "<td>+</td>"
        html_string += end_row

        for i in range(height):
            html_string += new_row
            html_string += f"<td>{i}</td>"
            for j in range(width):

                content = self.get_content((i, j))
                drawing = element_image[content[0]]

                html = '<td><img class="game" src=%s alt=""></img></td>' % drawing

                html_string += html
            html_string += f"<td>{i}</td>"
            html_string += end_row
        html_string += new_row
        html_string += "<td>+</td>"
        for j in range(width):
            html_string += f"<td style='text-align: center;'>{j}</td>"
        html_string += "<td>+</td>"
        html_string += end_row

        html_string += "</table>"
        return html_string


def miniMax(state: Tablerolinja, currentLevel: int, maxLevel: int, player: int, alpha: int, beta: int, stop: bool) -> Tuple[Tablerolinja, int, bool]:
    """Implementación del algoritmo minimax

    Args:
        state (Tablerolinja): Una instancia del tablero
        currentLevel (int): El nivel actual del arbol de minimax
        maxLevel (int): El máximo nivel al que se quiere llegar
        player (int): El jugador que queremos maximizar
        alpha (int): EL valor de alpha
        beta (int): El valor de beta
        stop (bool): Si se desea parar el algoritmo

    Returns:
        Tuple[Tablerolinja, int, bool]: Devuleve un Tablero, el coste y si se quiere parar
    """
    matriz = state.getMatrix()
    successorMatrices = []
    if player == 1:
        av = 1
    elif player == 2:
        av = -1
    if (not state.moveCanBeMade(player) or currentLevel == maxLevel):
        return (state.matrix, state.utility(), stop)

    # Actualizo el conjunto de las matrices de sucesores
    movimientos1 = []
    for row in range(0, len(state.matrix)):
        for col in range(0, len(state.matrix[row])):
            if state.matrix[row][col] == player:
                matrixTem = state.obtenerMov(
                    row, col, player, 1, 1)
                if matrixTem != matriz:
                    tupla = (matrixTem, state.getNFichasEnFila(row+av)+1)
                    movimientos1.append(tupla)
                    break
    for mov1 in movimientos1:
        matriz = mov1[0]
        navance = mov1[1]
        for row in range(0, len(matriz)):
            for col in range(0, len(matriz[row])):
                if matriz[row][col] == player:
                    temp = Tablerolinja(matriz)
                    temp.turno = state.turno
                    temp.nmov = 2
                    matrixTem = temp.obtenerMov(
                        row, col, player, 2, navance)
                    if matrixTem != matriz:
                        successorMatrices.append(matrixTem)
                        break

    if len(successorMatrices) == 0:
        stopDigging = True
        coste = state.utility()
        return (state.matrix, coste, stopDigging)

    bestMatrix = None

    if player == 2:
        maxValue = -math.inf  # alpha
        for i in range(0, len(successorMatrices)):
            mat = Tablerolinja(successorMatrices[i])
            matrizS, utility, stop = miniMax(
                mat, currentLevel + 1, maxLevel, 2, alpha, beta, stop)
            best = utility
            if best > maxValue:
                maxValue = best
                bestMatrix = mat.getMatrix()
            alpha = max(alpha, best)
            if best >= beta:
                return (matrizS, best, stop)
    else:
        minValue = math.inf  # beta
        for i in range(0, len(successorMatrices)):
            mat = Tablerolinja(successorMatrices[i])
            matrizS, utility, stop = miniMax(
                mat, currentLevel + 1, maxLevel, 1, alpha, beta, stop)
            if utility < minValue:
                minValue = utility
                bestMatrix = mat.getMatrix()
            beta = min(beta, utility)
            if utility <= alpha:
                return (matrizS, utility, stop)
    return (bestMatrix, utility, stop)


def performActionMinMax(state: Tablerolinja, player: int) -> Tablerolinja:
    """Hace la llamada al método minimax correspondiente con sus atributos

    Args:
        state (Tablerolinja): Una instancia del tablero
        player (int): El jugador que se quiere maximizar la puntuación

    Returns:
        Tablerolinja: El tablero depués de aplicar minimax
    """
    matrizB = state.getMatrix()
    tmpMatriz = [row[:] for row in matrizB]
    # =====================
    # Valor de profundidad:

    depth = 2

    matrizoptima = tmpMatriz
    stop = False
    currentLevel = 0

    tmpMatrizB = Tablerolinja(tmpMatriz)
    tmpMatrizB.turno = player
    if player == 1:
        (matrizoptima, valoroptimo, stop) = miniMax(tmpMatrizB,
                                                    currentLevel, depth, player, -math.inf, math.inf, stop)
    else:
        (matrizoptima, valoroptimo, stop) = miniMaxInv(tmpMatrizB,
                                                       currentLevel, depth, player, -math.inf, math.inf, stop)
    bestState = Tablerolinja(matrizoptima)
    return bestState


def miniMaxInv(state: Tablerolinja, currentLevel: int, maxLevel: int, player: int, alpha: int, beta: int, stop: bool) -> Tuple[Tablerolinja, int, bool]:
    """Implementación del algoritmo minimax, en el que seimplementa para el otro jugador para que la maquina pueda jugar contra otra maquina

    Args:
        state (Tablerolinja): Una instancia del tablero
        currentLevel (int): El nivel actual del arbol de minimax
        maxLevel (int): El máximo nivel al que se quiere llegar
        player (int): El jugador que queremos maximizar
        alpha (int): EL valor de alpha
        beta (int): El valor de beta
        stop (bool): Si se desea parar el algoritmo

    Returns:
        Tuple[Tablerolinja, int, bool]: Devuleve un Tablero, el coste y si se quiere parar
    """
    matriz = state.getMatrix()
    successorMatrices = []
    if player == 1:
        av = 1
    elif player == 2:
        av = -1
    if (not state.moveCanBeMade(player) or currentLevel == maxLevel):
        return (state.matrix, -state.utility(), stop)
    movimientos1 = []
    for row in range(0, len(state.matrix)):
        for col in range(0, len(state.matrix[row])):
            if state.matrix[row][col] == player:
                matrixTem = state.obtenerMov(
                    row, col, player, 1, 1)
                if matrixTem != matriz:
                    tupla = (matrixTem, state.getNFichasEnFila(row+av)+1)
                    movimientos1.append(tupla)
                    break
    movimientos1.reverse()
    for mov1 in movimientos1:
        matriz = mov1[0]
        navance = mov1[1]
        for row in range(0, len(matriz)):
            for col in range(0, len(matriz[row])):
                if matriz[row][col] == player:
                    temp = Tablerolinja(matriz)
                    temp.turno = state.turno
                    temp.nmov = 2
                    matrixTem = temp.obtenerMov(
                        row, col, player, 2, navance)
                    if matrixTem != matriz:
                        successorMatrices.append(matrixTem)
                        break
    successorMatrices.reverse()

    if len(successorMatrices) == 0:
        stopDigging = True
        coste = state.utility()
        return (state.matrix, coste, stopDigging)
    bestMatrix = None
    if player == 1:
        maxValue = -math.inf  # alpha
        for i in range(0, len(successorMatrices)):
            mat = Tablerolinja(successorMatrices[i])
            matrizS, utility, stop = miniMaxInv(
                mat, currentLevel + 1, maxLevel, 1, alpha, beta, stop)
            best = utility
            if best > maxValue:
                maxValue = best
                bestMatrix = mat.getMatrix()
            alpha = max(alpha, best)
            if best >= beta:
                return (matrizS, best, stop)
    else:
        minValue = math.inf  # beta
        for i in range(0, len(successorMatrices)):
            mat = Tablerolinja(successorMatrices[i])
            matrizS, utility, stop = miniMaxInv(
                mat, currentLevel + 1, maxLevel, 2, alpha, beta, stop)

            if utility < minValue:
                minValue = utility
                bestMatrix = mat.getMatrix()
            beta = min(beta, utility)
            if utility <= alpha:
                return (matrizS, utility, stop)
    return (bestMatrix, utility, stop)

### Implementación del juego de Linja en el que el jugador puede jugar contra la máquina
Primero será el turno del jugador, (fichas negras), en el que le pedira una fila, una columna que corresponderán a las coordenadas de la ficha que se quiere mover y luego pedirá una columna, en la que se colocará la ficha que se ha selecionado previamente al hacer el movimiento correspondiente, de igual manera se hará el segundo movimiento.
Después de esto será el turno del ordenador (fihas rojas), en el que aplicará minimax para realizar su movimientos y dará paso al usuario con un nuevo turno.
Si se quiere parar el juego, basta en introducir la palabra salir en cualquiera de las entradas por teclado.

In [ ]:
tableroInicial = [["N", "N", "N", "N", "N", "N"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["R", "R", "R", "R", "R", "R"]]

juego = Tablerolinja(tableroInicial)
juego.codificar()
salir = None
while (salir != "salir" and juego.fin() != True):
    salir = juego.entrada()
print("El ganador es: ", juego.obtenerGanador())
print("Estado final del tablero:")
display(HTML(juego.get_html()))

### Implementación de minimax en el que juega el ordenador vs el ordenador
En esta parte se ha creado una versión del juego que consiste en que la máquina va a ser los 2 jugadores, jugando uno contra otro.
Tiene 2 modos de ejecución, uno automático, en el que se intruducirán los segundos deseados entre cada impresión del tablero aunque cada movimiento queda grabado en la pantalla como una nueva imagen, en cambio, en el paso a paso después de cada movimiento pedirá una entrada al usuario para poder continuar, se puede introducir cualquier cosa aunque lo recomendado es un enter. Si se introduce salir parará la ejecución.

En las pruebas realizadas con profundidad 2 un juego completo suele tardar 10 segundos aproximadamente.

In [ ]:
tableroInicial = [["N", "N", "N", "N", "N", "N"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["N", "V", "V", "V", "V", "R"],
                  ["R", "R", "R", "R", "R", "R"]]

juego = Tablerolinja(tableroInicial)
juego.codificar()
salir = None
tiempodeseado = 0
opcion = input("Ejecutar automatico (1) o Ejecucion paso a paso(2)")
if opcion == "1":
    tiempodeseado = float(input(
        "Introduzca el numero de segundos minimo entre cada impresión (reomendado más de 0.5)"))
while (salir != "salir" and juego.fin() != True):
    inicio_tiempo = time()
    salir = juego.entradaAuto()
    tiempo_ejecucion = time() - inicio_tiempo
    if opcion == "2":
        salir = input(
            "salir para  salir de la ejecucion, cualquier otra cosa para continuar")
        clear_output()
    if tiempo_ejecucion < tiempodeseado:
        sleep(tiempodeseado-tiempo_ejecucion)

print("El ganador es: ", juego.obtenerGanador())
print("Estado final del tablero:")
display(HTML(juego.get_html()))